In [1]:
import csv
import re
import operator
import Levenshtein

train = 'train.csv'
test =  'test.csv'

def read_data(path, cols,ignore_header=True):
    csv_file_object = csv.reader(open(path, 'rt'))
    if ignore_header:
        header = next(csv_file_object)
    x = []
    #cnt = 0
    for row in csv_file_object:
        r = []
        #cnt = cnt +1
        for col in cols:
            r.append(row[col])
        x.append(r)
        #if cnt>100:
            #break
        
    return x

def string_normalize(s):
    res = s.lower()
    res = res.replace(' ', '')
    res = ''.join(c for c in res if c.isalnum())
    return res

def preprocess_data(raw_data, col):
    for row in raw_data:
        row[col] = string_normalize(row[col])

def best_match_key(keys, query):
    similarity = 0
    best_key = None
    for key in keys:
        sim = Levenshtein.ratio(key, query)
        if sim > similarity:
            similarity = sim
            best_key = key
    return (best_key, similarity)


In [2]:
def create_match(x, thresh=.85):
    match = {}
    for row in x:
        matched_key = None
        sku, query = row
        # Fuzzy matching.
        best_key, similarity = best_match_key(match.keys(), query)
        if similarity > thresh:
            matched_key = best_key
        else:
            match[query] = {sku : 1}
        if matched_key is None:
            continue
        if not 'match[matched_key]' in 'sku':
            match[matched_key][sku] = 1
        else:
            match[matched_key][sku] += 1
            
    for key in match.keys():
        tmp_dict = match[key]
        tmp_dict = sorted(tmp_dict.items(), key=operator.itemgetter(1))
        tmp_dict.reverse()
        match[key] = tmp_dict
    return match


In [3]:
def get_top(x):
    sku_count_dict = {}
    for row in x:
        if not 'sku_count_dict' in 'row[0]':
            sku_count_dict[row[0]] = 1
        else:
            sku_count_dict[row[0]] += 1
    sorted_dict = sorted(sku_count_dict.items(), key=operator.itemgetter(1))
    sorted_dict.reverse()

    res = []
    for i in range(len(sorted_dict)):
        res.append(sorted_dict[i][0])
    return res;


In [4]:
def predict(match, top, query, k=1, thresh=.7):
    res = []
    matched_key, similarity = best_match_key(match.keys(), query)
    # if similarity < 0.8:
    #     print 'matched_key = %s, query = %s, sim = %s' \
    #         % (matched_key, query, similarity)
    if similarity > thresh:
        for i in range(min(k, len(match[matched_key]))):
            res.append(match[matched_key][i][0])
    if len(res) < k:
        for i in range(len(top)):
            if top[i] not in res:
                res.append(top[i])
                if len(res) == k:
                    break
    return res


In [5]:
#accuracy
x = read_data(train, [1, 3])
preprocess_data(x, 1)

# Divides into training and cv.
l = int(len(x) * 1)
x_cv = x[l - 10 : :]
x = x[0 : l]

tp = 0
match = create_match(x)
p_cv = []
for row in x_cv:
    q = row[1]
    if row[0] in predict(match, get_top(x), q):
        tp += 1
accuracy = tp / len(x_cv)
print(accuracy)

0.3


In [6]:
if __name__ == '__main__':
    # Reads training data.
    print('Reading and preprocessing data...')
    x = read_data(train, [1, 3])
    preprocess_data(x, 1)

    # Divides into training and cv.
    l = int(len(x) * 1)
    x_cv = x[l - 10 : :]
    x = x[0 : l]

    top = get_top(x)

    # Predicts on cv.
    print('Predicting...')
    k = 5
    thresh_match = [0.75]
    thresh_predict = [0]
    best_match = None
    thresh1 = 0
    thresh2 = 0
    accuracy = -1
    for t1 in thresh_match:
        for t2 in thresh_predict:
            match = create_match(x, t1)
            p_cv = []
            for row in x_cv:
                q = row[1]
                p_cv.append(predict(match, top, q, k, t2))
            correct = 0.
            for i in range(len(x_cv)):
                if x_cv[i][0] in p_cv[i]:
                    correct += 1.
            ac = correct / len(x_cv)
            if ac > accuracy:
                accuracy = ac
                thresh1 = t1
                thresh2 = t2
                best_match = match
    print('thresh1 = %f, thresh2 = %f, accuracy = %f'\
        % (thresh1, thresh2, accuracy))

    # Reads test set.
    x_test = read_data(test, [2])
    preprocess_data(x_test, 0)

    # Predicts.
    res = []
    k = 5
    for row in x_test:
        q = row[0]
        res.append(predict(best_match, top, q, k, thresh2))

    open_file_object = csv.writer(open("result.csv", "wt"))
    open_file_object.writerow(['sku'])
    for p in res:
        open_file_object.writerow([' '.join(p)])
        print(p)

Reading and preprocessing data...
Predicting...
thresh1 = 0.750000, thresh2 = 0.000000, accuracy = 0.200000
['1617038', '1338089', '2628429', '8589244', '2722585']
['2670133', '2613542', '8589244', '2722585', '9977237']
['2541184', '2704058', '1011491', '2945052', '2808872']
['2095189', '8461915', '2823899', '3244621', '2856517']
['2945052', '9854804', '9927328', '3001046', '2095189']
['2945052', '9854804', '9927328', '3001046', '2095189']
['2541184', '2704058', '1011491', '2945052', '2808872']
['3244621', '2833031', '3184908', '2903172', '8589244']
['2945052', '1092494', '2633103', '2173065', '1228939']
['1930151', '8589244', '2722585', '9977237', '8947768']
['9713872', '9854507', '9854668', '8589244', '2722585']
['1179927', '2432276', '8589244', '2722585', '9977237']
['2032076', '2722585', '9927328', '9713872', '9261014']
['8709203', '8589244', '2722585', '9977237', '8947768']
['3071602', '2704058', '2945052', '3244621', '2856517']
['1515341', '8589244', '2722585', '9977237', '894776

['9927328', '1930151', '3455185', '2146641', '1251132']
['2670133', '2095189', '2749041', '2945052', '2671044']
['2173065', '2670133', '1228939', '2678578', '3447104']
['2541184', '2704058', '1011491', '2945052', '2808872']
['9902347', '2833031', '8589244', '2722585', '9977237']
['3088387', '2095189', '2340293', '9889193', '9854786']
['3088387', '2095189', '2340293', '9889193', '9854786']
['2375195', '2078113', '1199284', '3650081', '2842639']
['2095189', '8461915', '2823899', '3244621', '2856517']
['9854804', '2078113', '9345559', '2173065', '2737083']
['2842639', '2375195', '1470129', '2727253', '2704058']
['9976899', '1142357', '2078113', '1228939', '1989198']
['2842639', '2375195', '1470129', '2727253', '2704058']
['2541184', '2704058', '1011491', '2945052', '2808872']
['2051131', '8589244', '2722585', '9977237', '8947768']
['2541184', '2704058', '1011491', '2945052', '2808872']
['2628429', '1338089', '1251132', '2095189', '2670133']
['2842639', '2375195', '1470129', '2727253', '27

['2375195', '2078113', '1199284', '3650081', '2842639']
['2953607', '1617038', '2541184', '2375195', '2633103']
['9976899', '1142357', '2078113', '1228939', '1989198']
['9984142', '8589244', '2722585', '9977237', '8947768']
['9984142', '8589244', '2722585', '9977237', '8947768']
['3071408', '9984142', '3046603', '2173065', '9927328']
['2678108', '8589244', '2722585', '9977237', '8947768']
['9955514', '9889193', '2375195', '3046603', '2173065']
['9854804', '2613542', '2330703', '2467183', '2078113']
['3046603', '9854804', '7836737', '2432276', '2467183']
['2202037', '9927328', '1180104', '1161734', '9902347']
['9976899', '1142357', '2078113', '1228939', '1989198']
['2541184', '2823899', '2704058', '3244621', '2856544']
['2541184', '2704058', '1011491', '2945052', '2808872']
['1180104', '3650391', '2802508', '8589244', '2722585']
['2541184', '2704058', '1011491', '2945052', '2808872']
['2945052', '1092494', '2633103', '2173065', '1228939']
['9976899', '1142357', '2078113', '1228939', '19

['2977637', '2670133', '2703101', '2173065', '8589244']
['2977637', '2670133', '2703101', '2173065', '8589244']
['2541184', '2823899', '2704058', '3244621', '2856544']
['1179927', '2432276', '8589244', '2722585', '9977237']
['9854804', '2078113', '9345559', '2173065', '2737083']
['2541184', '2704058', '1011491', '2945052', '2808872']
['2856544', '2967045', '8589244', '2722585', '9977237']
['2945052', '1092494', '2633103', '2173065', '1228939']
['9949037', '9540428', '9889193', '2856544', '9701377']
['9976899', '9854804', '9902347', '2953607', '8589244']
['2856517', '8813313', '2541184', '1535584', '1563461']
['9331591', '8589244', '2722585', '9977237', '8947768']
['2032076', '2722585', '9927328', '9713872', '9261014']
['2856517', '2749041', '8589244', '2722585', '9977237']
['2953607', '2107458', '2670133', '2032076', '2856517']
['9976899', '1142357', '2078113', '1228939', '1989198']
['1404415', '1228939', '3244621', '3447104', '9854804']
['9949037', '9540428', '9889193', '2856544', '97

['2945052', '1228939', '2802508', '2678241', '1032361']
['2541184', '2704058', '1011491', '2945052', '2808872']
['9976899', '2842639', '2633103', '9854804', '1685052']
['9854507', '9854668', '8589244', '2722585', '9977237']
['2330703', '2095189', '3447104', '8589244', '2722585']
['9976899', '1142357', '2078113', '1228939', '1989198']
['2945052', '1092494', '2633103', '2173065', '1228939']
['2953607', '1251132', '2375195', '9854804', '2833031']
['9854804', '2613542', '2330703', '2467183', '2078113']
['9854804', '2613542', '2330703', '2467183', '2078113']
['2541184', '2704058', '1011491', '2945052', '2808872']
['9390358', '2833031', '8770868', '9902347', '2945052']
['2613542', '9936089', '9902347', '9854507', '2953607']
['9713872', '2633103', '9522634', '1997154', '3046066']
['2945052', '2628429', '3519923', '9461183', '8589244']
['2375195', '9976899', '2749041', '2670133', '2032076']
['9215333', '9331591', '9827204', '9936089', '1207275']
['2833031', '9902347', '8589244', '2722585', '99

...

['2541184', '2823899', '2704058', '3244621', '2856544']
['2953607', '1251132', '2375195', '9854804', '2833031']
['3088387', '2095189', '2340293', '9889193', '9854786']
['2953607', '1251132', '2375195', '9854804', '2833031']
['2945052', '1092494', '2633103', '2173065', '1228939']
['2095189', '8461915', '2823899', '3244621', '2856517']
['9976899', '1142357', '2078113', '1228939', '1989198']
['2945052', '1092494', '2633103', '2173065', '1228939']
['3559384', '1012721', '8589244', '2722585', '9977237']
['2953607', '1617038', '2541184', '2375195', '2633103']
['2953607', '1617038', '2541184', '2375195', '2633103']
['2953607', '1617038', '2541184', '2375195', '2633103']
['1563392', '3455398', '2032076', '1208468', '1199284']
['8760398', '2375195', '2107458', '2173065', '2670133']
['3046603', '9854804', '7836737', '2432276', '2467183']
['1012721', '3559384', '8589244', '2722585', '9977237']
['2541184', '2704058', '1011491', '2945052', '2808872']
['2953607', '1251132', '2375195', '9854804', '28

['2897116', '9902347', '1180104', '1580049', '2432276']
['2541184', '2704058', '1011491', '2945052', '2808872']
['2541184', '2704058', '1011491', '2945052', '2808872']
['2095189', '8461915', '2823899', '3244621', '2856517']
['2945052', '1092494', '2633103', '2173065', '1228939']
['9976899', '1142357', '2078113', '1228939', '1989198']
['2541184', '2823899', '2704058', '3244621', '2856544']
['2541184', '2704058', '1011491', '2945052', '2808872']
['2202037', '9927328', '1180104', '1161734', '9902347']
['9927328', '1930151', '3455185', '2146641', '1251132']
['2541184', '2467129', '9460736', '2375195', '2807036']
['2467183', '8736601', '2212043', '3433032', '3432088']
['3046603', '9854804', '7836737', '2432276', '2467183']
['2945052', '1092494', '2633103', '2173065', '1228939']
['2945052', '1092494', '2633103', '2173065', '1228939']
['2173065', '9854804', '2953816', '7959042', '9328809']
['2173065', '9854804', '2953816', '7959042', '9328809']
['2953607', '1251132', '2375195', '9854804', '28

['9927328', '1930151', '3455185', '2146641', '1251132']
['2541184', '9854804', '2375195', '2467183', '2173065']
['2945052', '9791723', '2340293', '8589244', '2722585']
['8760398', '2375195', '2107458', '2173065', '2670133']
['1404415', '1228939', '3244621', '3447104', '9854804']
['2938114', '9701377', '2945052', '2598588', '2173065']
['9902347', '2802508', '2670133', '2945052', '8589244']
['2541184', '2704058', '1011491', '2945052', '2808872']
['2938114', '9701377', '2945052', '2598588', '2173065']
['1404415', '1228939', '3244621', '3447104', '9854804']
['9976899', '2842639', '2633103', '9854804', '1685052']
['2842757', '8589244', '2722585', '9977237', '8947768']
['9955514', '9889193', '2375195', '3046603', '2173065']
['2945052', '1092494', '2633103', '2173065', '1228939']
['1011491', '9967476', '3447104', '1475036', '1199284']
['9345559', '2628429', '2107458', '9936089', '1617038']
['2467183', '2095189', '2758085', '2541184', '3447104']
['2541184', '2704058', '1011491', '2945052', '28

['2095189', '8461915', '2823899', '3244621', '2856517']
['2977637', '8589244', '2722585', '9977237', '8947768']
['2670133', '2897116', '1563392', '2935092', '2598588']
['1228939', '3071408', '2758085', '2802508', '9701377']
['2897055', '2823899', '8589244', '2722585', '9977237']
['2670133', '8947205', '9854804', '9980886', '2375195']
['3244621', '2833031', '3184908', '2903172', '8589244']
['3088387', '3552327', '8589244', '2722585', '9977237']
['2541184', '2704058', '1011491', '2945052', '2808872']
['2953607', '1617038', '2541184', '2375195', '2633103']
['2032076', '2722585', '9927328', '9713872', '9261014']
['3508275', '3668336', '2107458', '2173065', '3559384']
['1617038', '1338089', '2628429', '8589244', '2722585']
['2613542', '9936089', '9902347', '9854507', '2953607']
['2095189', '8461915', '2823899', '3244621', '2856517']
['2670133', '2897116', '1563392', '2935092', '2598588']
['9902347', '9984142', '2467183', '9487067', '2613542']
['2541184', '2823899', '2704058', '3244621', '28

['2670133', '9825676', '9345559', '9276473', '8589244']
['9854804', '2078113', '9345559', '2173065', '2737083']
['3001046', '2953607', '2107458', '8589244', '2722585']
['2953607', '1251132', '2375195', '9854804', '2833031']
['2749041', '1052221', '3088387', '8589244', '2722585']
['2670133', '9825676', '9345559', '9276473', '8589244']
['2670133', '9825676', '9345559', '9276473', '8589244']
['2541184', '9854804', '2375195', '2467183', '2173065']
['2541184', '2704058', '1011491', '2945052', '2808872']
['2953607', '1251132', '2375195', '9854804', '2833031']
['2095189', '8461915', '2823899', '3244621', '2856517']
['2107458', '1776209', '8589244', '2722585', '9977237']
['2633103', '1011067', '1431386', '1778119', '9406243']
['9949037', '9540428', '9889193', '2856544', '9701377']
['9976899', '1142357', '2078113', '1228939', '1989198']
['2541184', '2704058', '1011491', '2945052', '2808872']
['2095189', '8461915', '2823899', '3244621', '2856517']
['1179927', '2095189', '9713872', '9854804', '99